In [1]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
from skimage import transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn import model_selection
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
class MudraDataset(Dataset):
    def __init__(self, root_dir, state='All', transform=None):
        #"""
        #Args:
            #csv_files (string): Path to the csv file with annotations
            #root_dir (string): Directory with all the images
            #transform (callable, optional): Optional transform to be applied on a sample
        #"""
        #self.mudraJoints = pd.read_csv(csv_file)
        self.state = state
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.X_data = []
        self.Y_data = []
        
        if os.path.isdir(root_dir):
            for dirs in os.listdir(root_dir):
                #print(dirs)
                full_path = os.path.join(root_dir, dirs)
                csv_path = os.path.join(full_path, 'Annotation.csv')
                csv_file = pd.read_csv(csv_path)
                for i, row in csv_file.iterrows():
                    rlen = len(row)
                    img_path = os.path.join(full_path, row[0])
                    #print(img_path)
                    image = img_path
                    self.X_data.append(image)
                    jointAngle = row[1:rlen-1]
                    
                    jointAngle = np.array([jointAngle])
                    jointAngle = jointAngle.astype('float')
                    self.Y_data.append(jointAngle)
                    #sample = {'image': image, 'jointAngle': jointAngle}
                    #self.data.append(sample)

        X_train, X_test, Y_train, Y_test = model_selection.train_test_split(self.X_data,
                                                self.Y_data, test_size=0.25, random_state=42)
        if state == 'Train':    
            self.X_data, self.Y_data = X_train, Y_train
        elif state == 'Test':
            self.X_data, self.Y_data = X_test, Y_test
    
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        #img_name = os.path.join(self.root_dir, self.mudraJoints.iloc[idx, 0])
        #image = cv2.imread(img_name)
        #jointCood = self.mudraJoints.iloc[idx, 1:]
        #jointCood = np.array([jointCood])
        #jointCood = jointCood.astype('float')#.reshape(-1,2)
        #print(self.X_data[i])
        #print(self.X_data[idx])
        sample = {'image': cv2.imread(self.X_data[idx]).astype('float'), 'label': self.Y_data[idx]}
        
        if self.transform:
            sample = self.transform(sample)
        return sample

In [4]:
class TouchDataset(Dataset):
    def __init__(self, root_dir, num_classes, state='All', transform=None):
        #"""
        #Args:
            #csv_files (string): Path to the csv file with annotations
            #root_dir (string): Directory with all the images
            #transform (callable, optional): Optional transform to be applied on a sample
        #"""
        #self.mudraJoints = pd.read_csv(csv_file)
        self.state = state
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.X_data = []
        self.Y_data = []
        #self.Z_data = []
        self.pos = np.zeros(num_classes)
        self.total = 0
        
        
        if os.path.isdir(root_dir):
            for dirs in os.listdir(root_dir):
                #print(dirs)
                full_path = os.path.join(root_dir, dirs)
                csv_path = os.path.join(full_path, 'Annotation.csv')
                csv_file = pd.read_csv(csv_path)
                for i, row in csv_file.iterrows():
                    labels = np.zeros(num_classes)
                    rlen = len(row)
                    img_path = os.path.join(full_path, row[0])
                    #print(img_path)
                    image = img_path
                    self.X_data.append(image)
                    touchVal = row[-1]
                    #print(touchVal)
                    touchVal = touchVal[1:-1]
                    #print(touchVal)
                    touchLabel = touchVal.split(",")
                    for s in range(len(touchLabel)):
                        #print(touchLabel[s])
                        labels[int(touchLabel[s])] = 1
                        self.pos[int(touchLabel[s])] += 1
                        
                    #labels = np.array([jointAngle])
                    labels = labels.astype('float')
                    self.Y_data.append(labels)
                    #self.Z_data.append(row[0])
                    self.total += 1
                    #sample = {'image': image, 'jointAngle': jointAngle}
                    #self.data.append(sample)

        X_train, X_test, Y_train, Y_test = model_selection.train_test_split(self.X_data,
                                                self.Y_data, test_size=0.25, random_state=42)
        if state == 'Train':    
            self.X_data, self.Y_data = X_train, Y_train
        elif state == 'Test':
            self.X_data, self.Y_data = X_test, Y_test
        #print(self.Y_data)
    
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        #img_name = os.path.join(self.root_dir, self.mudraJoints.iloc[idx, 0])
        #image = cv2.imread(img_name)
        #jointCood = self.mudraJoints.iloc[idx, 1:]
        #jointCood = np.array([jointCood])
        #jointCood = jointCood.astype('float')#.reshape(-1,2)
        #print(self.X_data[i])
        #print(self.X_data[idx])
        sample = {'image': cv2.imread(self.X_data[idx]).astype('float'), 'label': self.Y_data[idx], 'name': self.X_data[idx]}
        
        if self.transform:
            sample = self.transform(sample)
        return sample

In [5]:
#Data Preprocessing Rescale, RandomCrop, and ToTensor
class Rescale(object):
    #Args:
    #output_size (tuple or int): Desired output size. If tuple, output is matched to output size. It int, smaller of
    #image edges is matched to output_size keeping aspect ratio the same
    
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
    
    def __call__(self, sample):
        image, joints, name = sample['image'], sample['label'], sample['name']
        #print(image)
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        
        new_h, new_w = int(new_h), int(new_w)
        
        img = transform.resize(image, (new_h, new_w))
        
        return {'image': img, 'label': joints, 'name': name}
    
class RandomCrop(object):
    #Args:
    #output_size (tuple or int): Desired output size. If int, square crop is made

    def __init__(self, output_size):
        assert isinstance(output_size, (tuple, int))

        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, joints = sample['image'], sample['label']
        #print(image)
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h, left: left + new_w]

        return {'image': image, 'label': joints}

class ToTensor(object):
    #Convert numpy array image to tensor image

    def __call__(self, sample):
        image, joints, name = sample['image'], sample['label'], sample['name']

        #swap color axis because
        #numpy image: H * W * C
        #torch image: C * H * W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).to(torch.float), 'label': torch.from_numpy(joints).to(torch.float), 'name': name}

In [6]:
def data_loader(dataset, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):
    num_train = len(dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
    
    if test:
        return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        return (train_loader, valid_loader)

In [7]:
train_dataset = TouchDataset(root_dir='Mudra Dataset/Single hand', num_classes=160,
                                  transform=transforms.Compose([Rescale((227,227)), ToTensor()]))
print(len(train_dataset))

13908
